In [1]:
import numpy as np
import pandas as pd 




N = 100
D = 10
K = 6*D



def Q(d,k):
    a = np.zeros((d+1,k+1))

    for _d in range(d+1):
        for _k in range(k+1):
            if (_k > 6*_d) or (_k < 2*_d):
                a[_d][_k] = 0.0
                continue
            if(_d == 1):
                a[_d][_k] = 1/5
                continue
            s = 0
            for i in range(2,7):
                if(_k>i):
                    s += a[_d-1][_k-i]

            a[_d][_k] = (1/5) * s

    return a[d][k]

def P(d,k):
    if k == 1:
        return 1.0 - (5.0/6.0)**d
    if (k > 6*d) or (k > 1 and k < 2*d):
        return 0.0
    return Q(d,k)*(5.0/6.0)**d


def generate_P_matrix(D):
	P_matrix = np.zeros((D+1,6*D+1))
	for i in range(1,D+1):
		for j in range(6*D+1):
			P_matrix[i][j] = P(i,j)
	return P_matrix




def EP(D=10):
    return 1+np.argmax(np.array([4*d*np.power(5/6,d) + (1-np.power(5/6,d)) for d in range(1,D+1) ]))




def G():
    def f(j,k):
        return (k > 0 and j>0)

    t = N + 6*D
    t = N
    a = np.zeros( ( t, t , 2 ))
    for i in range(1,t):
        for  j in range(1,t):
            l = [ probabilite[d][1]*(1 - a[j][i-1][0] ) + sum([ probabilite[d][k] * ( 1-a[j][i-k][0] * f(j,i-k) )
            for k in range (2*d,6*d + 1) ])  for d in range(1,D+1)]
            a[i][j][0] = max(l)
            a[i][j][1] = 1+np.argmax(l)

    return a



def esperance_gain():
    a = np.full((N+6*D,N+6*D),0)

    a[N: , :N] = 1
    a[:N , N: ] = -1

    optimal = np.zeros((N, N), dtype = int)

    for x in range(N - 1, -1, -1):
        for y in range(x, -1, -1):
            for i,j in {(x,y),(y,x)}:
                tmp = - probabilite[ :, 1: ].dot(a[j, (i + 1):(i + 6 * D + 1)])

                optimal[i, j] = tmp[1: ].argmax() + 1

                a[i, j] =  tmp[optimal[i, j]]

    return a,optimal



def matrice_gain(D=3):
    def f(i,j):
        if(i > j) : return 1
        if(j > i) : return -1
        if(i == j) : return 0
    
    
    return np.array([ [ f(j,i)  for  i  in range(6*D) ] for  j  in range(6*D) ] )



In [2]:
probabilite = generate_P_matrix(D)
p = pd.DataFrame(data=probabilite[1:,1:],index=["d="+str(i) for i in np.arange(1,D+1)],columns= ["k="+str(i) for i in np.arange(1,6*D+1)] )
p.to_csv('./probabiltes.csv')
p.head()

,k=1,k=2,k=3,k=4,k=5,k=6,k=7,k=8,k=9,k=10,...,k=51,k=52,k=53,k=54,k=55,k=56,k=57,k=58,k=59,k=60
d=1,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d=2,0.305556,0.000000,0.000000,0.027778,0.055556,0.083333,0.111111,0.138889,0.111111,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d=3,0.421296,0.000000,0.000000,0.000000,0.000000,0.004630,0.013889,0.027778,0.046296,0.069444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d=4,0.517747,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000772,0.003086,0.007716,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d=5,0.598122,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
D=3
import itertools
a = np.arange(1,D+1)

In [4]:
D = 3
a = np.array([[ probabilite[1:,1:][i].dot(np.arange(1,10*6+1)) - probabilite[1:,1:][j].dot(np.arange(1,10*6+1))  
 
  for j in range(0,D) ] for i in range(0,D) ])
eg_1 = pd.DataFrame(data=a,index=np.arange(1,D+1),columns= np.arange(1,D+1) )
eg_1

,1,2,3
1,0.000000,-2.361111,-3.865741
2,2.361111,0.000000,-1.504630
3,3.865741,1.504630,0.000000


In [5]:

def EGsimuL(d1, d2):
    p = 0
    truc = 0

    ptable = probabilite[max(d1, d2)][0:6*1+1]

    for i1 in range(1, ptable.shape[0]):
        for i2 in range(1, ptable.shape[0]):
            truc += (ptable[d2, i2] / (max(d1, d2) * 6))
            if i2 < i1 and d1 != 0:
                p += (ptable[d1, i1] / (max(d1, d2) * 6))
            elif i2 > i1 and d2 != 0:
                p -= (ptable[d2, i2] / (max(d1, d2) * 6))

    print(truc)

    return p if abs(p) > 0.01 else 0

In [6]:
#EGsimuL(2,3)

probabilite[1][1:6*1+1]

array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667])

In [7]:
a = np.dot(probabilite[1:D+1,1:6*D+1],np.arange(1,6*D+1))
a

array([3.5       , 5.86111111, 7.36574074])

In [8]:
import matplotlib.pyplot as plt
plt.plot(np.arange(0,d),a)
plt.show()

NameError: name 'd' is not defined

In [ ]:
import numpy as np

p = np.random.random(D)
p = p/p.sum()

q = np.random.random(D)
q = q/q.sum()

In [9]:
p

,k=1,k=2,k=3,k=4,k=5,k=6,k=7,k=8,k=9,k=10,...,k=51,k=52,k=53,k=54,k=55,k=56,k=57,k=58,k=59,k=60
d=1,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=2,0.305556,0.000000,0.000000,0.027778,0.055556,0.083333,0.111111,0.138889,0.111111,0.083333,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=3,0.421296,0.000000,0.000000,0.000000,0.000000,0.004630,0.013889,0.027778,0.046296,0.069444,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=4,0.517747,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000772,0.003086,0.007716,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=5,0.598122,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000129,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=6,0.665102,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=7,0.720918,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=8,0.767432,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=9,0.806193,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000016,0.000004,8.930613e-07,9.922903e-08,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
d=10,0.838494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000686,0.000366,1.801007e-04,8.111973e-05,0.000033,0.000012,0.000004,9.095994e-07,1.653817e-07,1.653817e-08


In [10]:
q

NameError: name 'q' is not defined

In [11]:
g = matrice_gain(D)

In [12]:
g.shape

(18, 18)

In [13]:
def q(d, k):
    if k < 2 * d or k > 6 * d:
        return 0

    if d == 1:
        return 1 / 5

    sum_probability = 0

    for i in range(2, 6 + 1):
        sum_probability += q(d - 1, k - i) / 5

    return sum_probability


def p(d, k):
    if k == 1:
        return 1 - (5 / 6) ** d
    elif k < 2 * d or k > 6 * d:
        return 0
    else:
        return ((5 / 6) ** d) * q(d, k)


def p_table(D):
    table = np.zeros((D + 1, 6 * D + 1))
    for d in range(1, D + 1):
        for k in range(1, 6 * D + 1):
            table[d, k] = p(d, k)
    np.savetxt('test.out', table, delimiter=',')
    return table


In [14]:
def EGsimuL(d1, d2):
    p = 0
    truc = 0

    ptable = p_table(max(d1, d2))

    for i1 in range(1, ptable.shape[1]):
        for i2 in range(1, ptable.shape[1]):
            
            truc += (ptable[d2, i2] / (max(d1, d2) * 6))
            
            if i2 < i1 and d1 != 0:
                p += (ptable[d1, i1] / (max(d1, d2) * 6))
            
            elif i2 > i1 and d2 != 0:
                p -= (ptable[d2, i2] / (max(d1, d2) * 6))
    
    return p if abs(p) > 0.0000001 else 0

In [15]:
D=3
np.array([ [ EGsimuL(i,j) for i in range(1,D+1) ] for j in range(1,D+1)])

array([[ 0.        ,  0.19675926,  0.21476337],
       [-0.19675926,  0.        ,  0.08359053],
       [-0.21476337, -0.08359053,  0.        ]])

In [16]:
def eg(D=3):
    def f(d1,d2):
        s_1 = s_2 = 0
        n = max(d1,d2)*6

        for i in range(1,6*d1+1):
            for j in range(1,6*d2+1):    

                if(j<i):
                    s_1 += probabilite[d1][i]

                if(i<j):
                    s_2 += probabilite[d2][j]
        p = (s_1 - s_2)/n
        return  np.round(p, 7)

    a = np.zeros((D+1,D+1))
    for d1 in range(1,D+1):
        for d2 in range(1,D+1):        
            a[d1][d2] = f(d1,d2)
    return a[1:,1:]


eg()

array([[ 0.       , -0.1157407, -0.0537551],
       [ 0.1157407,  0.       , -0.0655864],
       [ 0.0537551,  0.0655864,  0.       ]])

In [17]:
def EGsimuP(d1, d2 , pt):
    p_table_j1 = pt[d1]
    p_table_j2 = pt[d2]
    eg_j1=0
    
    for i in range(1, len(p_table_j1)):
        eg2=0
        for j in range(1, len(p_table_j2)):
            if i>j:
                eg2 += p_table_j2[j]
            if j>i:
                eg2 -= p_table_j2[j]
        eg_j1 += eg2  *p_table_j1[i]

    return eg_j1 


def EGmatP(D):
    mat = np.zeros((D + 1, D + 1))
    pt = p_table(D)

    for d1 in range(D + 1):
        for d2 in range(D + 1):
            mat[d1, d2] = EGsimuP(d1, d2, pt)

    return mat[1:,1:]

In [18]:
eg()

array([[ 0.       , -0.1157407, -0.0537551],
       [ 0.1157407,  0.       , -0.0655864],
       [ 0.0537551,  0.0655864,  0.       ]])

In [19]:
np.matrix(eg())

matrix([[ 0.       , -0.1157407, -0.0537551],
        [ 0.1157407,  0.       , -0.0655864],
        [ 0.0537551,  0.0655864,  0.       ]])

In [21]:
m = np.array([[0,-2,+1,+1],[+2,0,-2,+1],[-1,+2,0,-2],[-1,-1,+2,0]])
print(m)

[[ 0 -2  1  1]
 [ 2  0 -2  1]
 [-1  2  0 -2]
 [-1 -1  2  0]]


In [64]:
def multiply(v,m):
    s=str()
    for i in range(len(v)):
        for j in range(len(m[0])): 
            for k in range(len(m)):
                s 
    return s

In [69]:
c = [[ str(m[j][i]) for i in range(0,m.shape[1]) ] for j in range(0,m.shape[0])]
b = ['a','b','c','d']

In [70]:
multiply(c,b)

'0*a+-2*b+1*c+1*d+2*a+0*b+-2*c+1*d+-1*a+2*b+0*c+-2*d+-1*a+-1*b+2*c+0*d+'

In [76]:
[[ 0 for k in range(0,len(b))] for j in range(len(b[0])) ] # for i in range(len(c)) ] 

[[0, 0, 0, 0]]